In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def firsts(list):
    answer=[]
    for pair in list:
        answer=answer+[pair[0]]
    return answer

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jLeoPowerConstantTerm(m,power,multiplier):
    trun=jLeo(power,m,multiplier)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def last_element(listo):return listo[len(listo)-1]
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

print("okay1")

okay1


In [ ]:
#output snipped; I interrupted this after k = 94.
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [1..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst= jpowerConstantTerm(m,power)
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-nszfunction(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
#below was originally done in a separate cell
#import pickle
#wfile = open('/Users/barrybrent/26oct22no1.txt','wb')
#t = pickle.dumps(str(polydata))
#wfile.write(t)
#wfile.close()
---------------------------------------------------------------------------------------------------
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for power in [95..100]:
    start=time.time()
    data=[]
    for m in [3..3*power+3]:
        cst=QQ(jpowerConstantTerm(m,power))
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[power,rl]]
    nf=ZZ(numericalfactor(rl))
    difference=nf/24-broadhurst(power)
    tests=tests+[difference]
    finish=time.time()
    print("(power,difference,time): ",(power,difference,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/26oct22no2.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [2]:
a=[1,2,3];b=[4,5,6];c=a+b;print(c)

[1, 2, 3, 4, 5, 6]


In [6]:
import pickle
rsfile = open('/Users/barrybrent/26oct22no1.txt','rb')
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)

rtfile = open('/Users/barrybrent/26oct22no2.txt','rb')
rt=pickle.load(rtfile)
rtfile.close()
t=stripQuotationMarks(rt)

u=s+t

wufile = open('/Users/barrybrent/5dec22no1.txt','wb')
u = pickle.dumps(str(u))
wufile.write(u)

1968604

In [8]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [29]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            data=data+[ratio]
            #print((p,k,dspk,m,ratio))
ud=uniteList(data)
ud=sortAndSmash(ud)
print("prime: ",p," ratios:");print(ud)
print("==========================================================================================")
for rd in ud:
    print("prime: ",p," ratio:",rd," dspk: ",dspk)
    ld=len(data)  
    positions=[]
    for j in [0..ld-1]:
        if data[j]==rd:positions=positions+[j]
    print("first position:",positions[0])
    d=difference_table(positions)
    print("difference_table(positions):");print(d);
    print("---------------------------------------------------------------------------------------")

prime:  3  ratios:
[0, 1, 2, 3, 4, 5, 6, 7, 10, 13, 16, 19, 28, 37, 46, 55, 82, 109, 136, 163, 244, 325, 406]
prime:  3  ratio: 0  dspk:  1
first position: 5
difference_table(positions):
[1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 1, 2, 1, 1, 2, 2, 6, 2, 2, 1, 1, 1, 1, 7, 1, 2, 1, 1, 1, 2, 1, 6, 1, 1, 2, 1, 2, 1, 1, 6, 2, 1, 1, 1, 1, 3, 6, 1, 1, 3, 1, 1, 2, 6, 1, 1, 2, 1, 3, 7, 3, 2, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 2, 1, 1, 2, 1, 1, 6, 2, 1, 1, 1, 1, 1, 1, 7, 1, 3, 1, 1, 3, 6, 1, 1, 1, 1, 1, 2, 1, 1, 6, 2, 1, 2, 1, 2, 7, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 2, 3, 1, 6, 3, 1, 1, 1, 1, 8, 1, 1, 2, 1, 1, 1, 2, 6, 1, 4, 2, 1, 1, 6, 2, 1, 2, 1, 1, 2, 6, 1, 2, 2, 1, 2, 7, 1, 1, 1, 1, 1, 2, 2, 6, 2, 3, 1, 2, 7, 1, 1, 1, 2, 1, 1, 1, 1, 6, 1, 1, 1, 2, 4, 6, 4, 1, 1, 1, 1, 1, 6, 1, 1, 1, 2, 1, 1, 1, 1, 6, 1, 2, 1, 1, 2, 2, 6, 2, 3, 1, 1, 1, 1, 6, 1, 2, 1, 1, 1, 2, 1, 6, 1, 1, 3, 2, 1, 1, 6, 2, 1, 2, 1, 3, 6, 1, 1, 2, 1, 1, 1, 8, 1, 1, 2, 1, 3, 1, 6, 3, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1,

In [28]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=5
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            data=data+[ratio]
            #print((p,k,dspk,m,ratio))
ud=uniteList(data)
ud=sortAndSmash(ud)
print("prime: ",p," ratios:");print(ud)
print("==========================================================================================")
for rd in ud:
    print("prime: ",p," ratio:",rd," dspk: ",dspk)
    ld=len(data)  
    positions=[]
    for j in [0..ld-1]:
        if data[j]==rd:positions=positions+[j]
    print("first position:",positions[0])
    d=difference_table(positions)
    print("difference_table(positions):");print(d);
    print("---------------------------------------------------------------------------------------")

prime:  5  ratios:
[0, 1, 2, 3, 4, 8, 13, 18, 38, 63, 88]
prime:  5  ratio: 0  dspk:  1
first position: 0
difference_table(positions):
[1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 

In [30]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            data=data+[ratio]
            #print((p,k,dspk,m,ratio))
ud=uniteList(data)
ud=sortAndSmash(ud)
print("prime: ",p," ratios:");print(ud)
print("==========================================================================================")
for rd in ud:
    print("prime: ",p," ratio:",rd," dspk: ",dspk)
    ld=len(data)  
    positions=[]
    for j in [0..ld-1]:
        if data[j]==rd:positions=positions+[j]
    print("first position:",positions[0])
    d=difference_table(positions)
    print("difference_table(positions):");print(d);
    print("---------------------------------------------------------------------------------------")

prime:  7  ratios:
[0, 1, 2, 3, 4, 5, 12, 19, 33, 82, 131]
prime:  7  ratio: 0  dspk:  1
first position: 0
difference_table(positions):
[1, 1, 1, 1, 1, 1, 2, 10, 1, 2, 2, 1, 2, 1, 2, 2, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 5, 1, 1, 2, 1, 2, 1, 3, 1, 4, 6, 1, 3, 1, 2, 1, 1, 1, 1, 3, 6, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 3, 6, 1, 1, 1, 1, 2, 1, 3, 4, 1, 5, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 5, 2, 1, 1, 1, 1, 2, 1, 2, 1, 3, 1, 5, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 3, 6, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 3, 7, 3, 1, 1, 1, 1, 2, 1, 3, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 7, 1, 1, 1, 3, 2, 1, 4, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5, 2, 2, 1, 3, 2, 1, 1, 1, 3, 6, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 6, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 5, 2, 1, 1, 1, 1, 1

In [53]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if ratio==5:
                data=data+[mod(m,p)]
            #print((p,k,dspk,m,ratio))
print(data);print()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]



In [55]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if ZZ(mod(m,p))==0:data=data+[ratio]
            #print((p,k,dspk,m,ratio))
print(data);print()

[1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 3, 19, 131, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 2, 12, 82, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 1, 5, 33, 3, 19, 131, 1, 

In [62]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (ZZ(mod(m,p))==0)&(ratio==5):data=data+[order(p,m)]
            #print((p,k,dspk,m,ratio))
print(data);print()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [63]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (order(p,m)>1)&(ratio==5):data=data+[m]
            #print((p,k,dspk,m,ratio))
print(data);print()

[]



In [79]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (ratio==0):data=data+[order(p,m)]
            #print((p,k,dspk,m,ratio))
uld=uniteList(data)
print(uld)

[0]


In [82]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (order(p,m)==0):data=data+[ratio]
            #print((p,k,dspk,m,ratio))
uld=uniteList(data)
print(uld)

[0, 1, 2, 3, 4]


In [78]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=1
data=[]
for m in [3..2003]:
    mp=ZZ(mod(m,p))
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (ratio==1):data=data+[order(p,m)]
uld=uniteList(data)
print(uld)
dtu=difference_table(uld)
print();print(dtu)

[0, 1]

[1]


In [80]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
data=[]
for m in [3..2003]:
    mp=ZZ(mod(m,p))
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (ratio==0):data=data+[order(p,m)]
uld=uniteList(data)
print(uld)
dtu=difference_table(uld)
print();print(dtu)

[0]

[]


In [81]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
data=[]
for m in [3..2003]:
    mp=ZZ(mod(m,p))
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (order(p,m)==0):data=data+[ratio]
uld=uniteList(data)
print(uld)
dtu=difference_table(uld)
print();print(dtu)

[0, 1/2, 1, 3/2]

[1/2, 1/2, 1/2]


In [17]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
dspm=1
data=[]
for m in [3..2003]:
    sm=digitsum(p,m)
    if digitsum(p,m)==dspm:
        for h in [0..len(s)-1]:
            k=s[h][0]
            if digitsum(p,k)==dspk:
                poly=D(s[h][1])
                polym=evaluate(poly,m)
                o=order(p,polym)
                ratio=o/dspk
                if (order(p,m)==0):
                    if ratio==1/2:
                        print((k,m))
                        print(k.digits(p))
                        print(m.digits(p))
                        print("----------------------------------------------------------------------------")
# no output

In [18]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
dspm=3
data=[]
for m in [3..2003]:
    sm=digitsum(p,m)
    if digitsum(p,m)==dspm:
        for h in [0..len(s)-1]:
            k=s[h][0]
            if digitsum(p,k)==dspk:
                poly=D(s[h][1])
                polym=evaluate(poly,m)
                o=order(p,polym)
                ratio=o/dspk
                if (order(p,m)==0):
                    if ratio==1/2:
                        print((k,m))
                        print(k.digits(p))
                        print(m.digits(p))
                        print("----------------------------------------------------------------------------")

(50, 9)
[1, 0, 1]
[2, 1]
----------------------------------------------------------------------------
(56, 9)
[0, 1, 1]
[2, 1]
----------------------------------------------------------------------------


In [20]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
dspm=4
data=[]
for m in [3..2003]:
    sm=digitsum(p,m)
    if digitsum(p,m)==dspm:
        for h in [0..len(s)-1]:
            k=s[h][0]
            if digitsum(p,k)==dspk:
                poly=D(s[h][1])
                polym=evaluate(poly,m)
                o=order(p,polym)
                ratio=o/dspk
                if (order(p,m)==0):
                    print((k,m,ratio))
                    print(k.digits(p))
                    print(m.digits(p))
                    print("----------------------------------------------------------------------------")


(2, 4, 0)
[2]
[4]
----------------------------------------------------------------------------
(8, 4, 0)
[1, 1]
[4]
----------------------------------------------------------------------------
(14, 4, 0)
[0, 2]
[4]
----------------------------------------------------------------------------
(50, 4, 0)
[1, 0, 1]
[4]
----------------------------------------------------------------------------
(56, 4, 0)
[0, 1, 1]
[4]
----------------------------------------------------------------------------
(98, 4, 0)
[0, 0, 2]
[4]
----------------------------------------------------------------------------
(2, 10, 0)
[2]
[3, 1]
----------------------------------------------------------------------------
(8, 10, 1)
[1, 1]
[3, 1]
----------------------------------------------------------------------------
(14, 10, 0)
[0, 2]
[3, 1]
----------------------------------------------------------------------------
(50, 10, 0)
[1, 0, 1]
[3, 1]
---------------------------------------------------------------------

(98, 394, 0)
[0, 0, 2]
[2, 0, 1, 1]
----------------------------------------------------------------------------
(2, 400, 0)
[2]
[1, 1, 1, 1]
----------------------------------------------------------------------------
(8, 400, 0)
[1, 1]
[1, 1, 1, 1]
----------------------------------------------------------------------------
(14, 400, 0)
[0, 2]
[1, 1, 1, 1]
----------------------------------------------------------------------------
(50, 400, 0)
[1, 0, 1]
[1, 1, 1, 1]
----------------------------------------------------------------------------
(56, 400, 0)
[0, 1, 1]
[1, 1, 1, 1]
----------------------------------------------------------------------------
(98, 400, 0)
[0, 0, 2]
[1, 1, 1, 1]
----------------------------------------------------------------------------
(2, 442, 0)
[2]
[1, 0, 2, 1]
----------------------------------------------------------------------------
(8, 442, 0)
[1, 1]
[1, 0, 2, 1]
----------------------------------------------------------------------------
(14, 442

In [14]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
dspm=2
data=[]
for m in [3..2003]:
    sm=digitsum(p,m)
    if digitsum(p,m)==dspm:
        for h in [0..len(s)-1]:
            k=s[h][0]
            if digitsum(p,k)==dspk:
                poly=D(s[h][1])
                polym=evaluate(poly,m)
                o=order(p,polym)
                ratio=o/dspk
                if (order(p,m)==0):
                    if ratio==1/2:
                        print((k,m,dsm))
                        print(k.digits(p))
                        print(m.digits(p))
                        print("----------------------------------------------------------------------------")
# no output

In [4]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
data=[]
for j in [1..100]:
    m=1+p^j
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (order(p,m)==0):data=data+[ratio]
uld=uniteList(data)
print(uld)
dtu=difference_table(uld)
print();print(dtu)

[0]

[]


In [5]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
data=[]
for j in [1..100]:
    m=p*(1+p^j)
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (order(p,m)==0):data=data+[ratio]
uld=uniteList(data)
print(uld)
dtu=difference_table(uld)
print();print(dtu)

[]

[]


In [6]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=7
dspk=2
data=[]
for j in [1..100]:
    m=p^j*(1+p)
    for h in [0..len(s)-1]:
        k=s[h][0]
        if digitsum(p,k)==dspk:
            poly=D(s[h][1])
            polym=evaluate(poly,m)
            o=order(p,polym)
            ratio=o/dspk
            if (order(p,m)==0):data=data+[ratio]
uld=uniteList(data)
print(uld)
dtu=difference_table(uld)
print();print(dtu)

[]

[]


In [31]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
dspk=1
for h in [0..len(s)-1]:
        k=s[h][0]
        dspk=digitsum(p,k)
        lk=len(k.digits(p))
        poly=D(s[h][1])
        for m in [3..203]:
            lm=len(m.digits(p))
            dsm=digitsum(p,m)
            polym=evaluate(poly,m)
            o=order(p,polym)
            rk=o/dspk
            rm=o/dsm
            print((k,m,lk,lm,dsm,o,rk))
                

(1, 3, 1, 2, 1, 1, 1)
(1, 4, 1, 2, 2, 0, 0)
(1, 5, 1, 2, 3, 0, 0)
(1, 6, 1, 2, 2, 1, 1)
(1, 7, 1, 2, 3, 0, 0)
(1, 8, 1, 2, 4, 0, 0)
(1, 9, 1, 3, 1, 2, 2)
(1, 10, 1, 3, 2, 0, 0)
(1, 11, 1, 3, 3, 0, 0)
(1, 12, 1, 3, 2, 1, 1)
(1, 13, 1, 3, 3, 0, 0)
(1, 14, 1, 3, 4, 0, 0)
(1, 15, 1, 3, 3, 1, 1)
(1, 16, 1, 3, 4, 0, 0)
(1, 17, 1, 3, 5, 0, 0)
(1, 18, 1, 3, 2, 2, 2)
(1, 19, 1, 3, 3, 0, 0)
(1, 20, 1, 3, 4, 0, 0)
(1, 21, 1, 3, 3, 1, 1)
(1, 22, 1, 3, 4, 0, 0)
(1, 23, 1, 3, 5, 0, 0)
(1, 24, 1, 3, 4, 1, 1)
(1, 25, 1, 3, 5, 0, 0)
(1, 26, 1, 3, 6, 0, 0)
(1, 27, 1, 4, 1, 3, 3)
(1, 28, 1, 4, 2, 0, 0)
(1, 29, 1, 4, 3, 0, 0)
(1, 30, 1, 4, 2, 1, 1)
(1, 31, 1, 4, 3, 0, 0)
(1, 32, 1, 4, 4, 0, 0)
(1, 33, 1, 4, 3, 1, 1)
(1, 34, 1, 4, 4, 0, 0)
(1, 35, 1, 4, 5, 0, 0)
(1, 36, 1, 4, 2, 2, 2)
(1, 37, 1, 4, 3, 0, 0)
(1, 38, 1, 4, 4, 0, 0)
(1, 39, 1, 4, 3, 1, 1)
(1, 40, 1, 4, 4, 0, 0)
(1, 41, 1, 4, 5, 0, 0)
(1, 42, 1, 4, 4, 1, 1)
(1, 43, 1, 4, 5, 0, 0)
(1, 44, 1, 4, 6, 0, 0)
(1, 45, 1, 4, 3, 2, 2)
(1, 46, 1, 4, 4, 0

(5, 9, 2, 3, 1, 8, 8/3)
(5, 10, 2, 3, 2, 2, 2/3)
(5, 11, 2, 3, 3, 2, 2/3)
(5, 12, 2, 3, 2, 3, 1)
(5, 13, 2, 3, 3, 1, 1/3)
(5, 14, 2, 3, 4, 1, 1/3)
(5, 15, 2, 3, 3, 3, 1)
(5, 16, 2, 3, 4, 2, 2/3)
(5, 17, 2, 3, 5, 2, 2/3)
(5, 18, 2, 3, 2, 8, 8/3)
(5, 19, 2, 3, 3, 2, 2/3)
(5, 20, 2, 3, 4, 4, 4/3)
(5, 21, 2, 3, 3, 3, 1)
(5, 22, 2, 3, 4, 1, 1/3)
(5, 23, 2, 3, 5, 1, 1/3)
(5, 24, 2, 3, 4, 3, 1)
(5, 25, 2, 3, 5, 2, 2/3)
(5, 26, 2, 3, 6, 2, 2/3)
(5, 27, 2, 4, 1, 13, 13/3)
(5, 28, 2, 4, 2, 2, 2/3)
(5, 29, 2, 4, 3, 2, 2/3)
(5, 30, 2, 4, 2, 3, 1)
(5, 31, 2, 4, 3, 1, 1/3)
(5, 32, 2, 4, 4, 1, 1/3)
(5, 33, 2, 4, 3, 3, 1)
(5, 34, 2, 4, 4, 3, 1)
(5, 35, 2, 4, 5, 2, 2/3)
(5, 36, 2, 4, 2, 8, 8/3)
(5, 37, 2, 4, 3, 2, 2/3)
(5, 38, 2, 4, 4, 2, 2/3)
(5, 39, 2, 4, 3, 3, 1)
(5, 40, 2, 4, 4, 1, 1/3)
(5, 41, 2, 4, 5, 1, 1/3)
(5, 42, 2, 4, 4, 3, 1)
(5, 43, 2, 4, 5, 2, 2/3)
(5, 44, 2, 4, 6, 2, 2/3)
(5, 45, 2, 4, 3, 8, 8/3)
(5, 46, 2, 4, 4, 2, 2/3)
(5, 47, 2, 4, 5, 3, 1)
(5, 48, 2, 4, 4, 3, 1)
(5, 49, 2, 4, 5, 1, 1

(7, 56, 2, 4, 4, 1, 1/3)
(7, 57, 2, 4, 3, 3, 1)
(7, 58, 2, 4, 4, 1, 1/3)
(7, 59, 2, 4, 5, 1, 1/3)
(7, 60, 2, 4, 4, 3, 1)
(7, 61, 2, 4, 5, 4, 4/3)
(7, 62, 2, 4, 6, 1, 1/3)
(7, 63, 2, 4, 3, 10, 10/3)
(7, 64, 2, 4, 4, 1, 1/3)
(7, 65, 2, 4, 5, 1, 1/3)
(7, 66, 2, 4, 4, 3, 1)
(7, 67, 2, 4, 5, 2, 2/3)
(7, 68, 2, 4, 6, 2, 2/3)
(7, 69, 2, 4, 5, 3, 1)
(7, 70, 2, 4, 6, 1, 1/3)
(7, 71, 2, 4, 7, 1, 1/3)
(7, 72, 2, 4, 4, 10, 10/3)
(7, 73, 2, 4, 5, 1, 1/3)
(7, 74, 2, 4, 6, 2, 2/3)
(7, 75, 2, 4, 5, 3, 1)
(7, 76, 2, 4, 6, 1, 1/3)
(7, 77, 2, 4, 7, 1, 1/3)
(7, 78, 2, 4, 6, 3, 1)
(7, 79, 2, 4, 7, 1, 1/3)
(7, 80, 2, 4, 8, 1, 1/3)
(7, 81, 2, 5, 1, 24, 8)
(7, 82, 2, 5, 2, 1, 1/3)
(7, 83, 2, 5, 3, 1, 1/3)
(7, 84, 2, 5, 2, 3, 1)
(7, 85, 2, 5, 3, 1, 1/3)
(7, 86, 2, 5, 4, 1, 1/3)
(7, 87, 2, 5, 3, 3, 1)
(7, 88, 2, 5, 4, 2, 2/3)
(7, 89, 2, 5, 5, 1, 1/3)
(7, 90, 2, 5, 2, 10, 10/3)
(7, 91, 2, 5, 3, 1, 1/3)
(7, 92, 2, 5, 4, 1, 1/3)
(7, 93, 2, 5, 3, 3, 1)
(7, 94, 2, 5, 4, 2, 2/3)
(7, 95, 2, 5, 5, 2, 2/3)
(7, 96, 2, 5,

(10, 42, 3, 4, 4, 2, 1)
(10, 43, 3, 4, 5, 0, 0)
(10, 44, 3, 4, 6, 0, 0)
(10, 45, 3, 4, 3, 12, 6)
(10, 46, 3, 4, 4, 0, 0)
(10, 47, 3, 4, 5, 1, 1/2)
(10, 48, 3, 4, 4, 2, 1)
(10, 49, 3, 4, 5, 0, 0)
(10, 50, 3, 4, 6, 0, 0)
(10, 51, 3, 4, 5, 2, 1)
(10, 52, 3, 4, 6, 0, 0)
(10, 53, 3, 4, 7, 1, 1/2)
(10, 54, 3, 4, 2, 22, 11)
(10, 55, 3, 4, 3, 1, 1/2)
(10, 56, 3, 4, 4, 0, 0)
(10, 57, 3, 4, 3, 2, 1)
(10, 58, 3, 4, 4, 0, 0)
(10, 59, 3, 4, 5, 0, 0)
(10, 60, 3, 4, 4, 2, 1)
(10, 61, 3, 4, 5, 1, 1/2)
(10, 62, 3, 4, 6, 0, 0)
(10, 63, 3, 4, 3, 12, 6)
(10, 64, 3, 4, 4, 0, 0)
(10, 65, 3, 4, 5, 0, 0)
(10, 66, 3, 4, 4, 2, 1)
(10, 67, 3, 4, 5, 1, 1/2)
(10, 68, 3, 4, 6, 1, 1/2)
(10, 69, 3, 4, 5, 2, 1)
(10, 70, 3, 4, 6, 0, 0)
(10, 71, 3, 4, 7, 0, 0)
(10, 72, 3, 4, 4, 12, 6)
(10, 73, 3, 4, 5, 0, 0)
(10, 74, 3, 4, 6, 1, 1/2)
(10, 75, 3, 4, 5, 2, 1)
(10, 76, 3, 4, 6, 0, 0)
(10, 77, 3, 4, 7, 0, 0)
(10, 78, 3, 4, 6, 2, 1)
(10, 79, 3, 4, 7, 0, 0)
(10, 80, 3, 4, 8, 1, 1/2)
(10, 81, 3, 5, 1, 32, 16)
(10, 82, 3, 5, 2,

(12, 94, 3, 5, 4, 1, 1/2)
(12, 95, 3, 5, 5, 1, 1/2)
(12, 96, 3, 5, 4, 2, 1)
(12, 97, 3, 5, 5, 0, 0)
(12, 98, 3, 5, 6, 4, 2)
(12, 99, 3, 5, 3, 14, 7)
(12, 100, 3, 5, 4, 3, 3/2)
(12, 101, 3, 5, 5, 1, 1/2)
(12, 102, 3, 5, 4, 2, 1)
(12, 103, 3, 5, 5, 0, 0)
(12, 104, 3, 5, 6, 0, 0)
(12, 105, 3, 5, 5, 2, 1)
(12, 106, 3, 5, 6, 0, 0)
(12, 107, 3, 5, 7, 1, 1/2)
(12, 108, 3, 5, 2, 26, 13)
(12, 109, 3, 5, 3, 1, 1/2)
(12, 110, 3, 5, 4, 0, 0)
(12, 111, 3, 5, 3, 2, 1)
(12, 112, 3, 5, 4, 0, 0)
(12, 113, 3, 5, 5, 0, 0)
(12, 114, 3, 5, 4, 2, 1)
(12, 115, 3, 5, 5, 1, 1/2)
(12, 116, 3, 5, 6, 2, 1)
(12, 117, 3, 5, 3, 14, 7)
(12, 118, 3, 5, 4, 2, 1)
(12, 119, 3, 5, 5, 0, 0)
(12, 120, 3, 5, 4, 2, 1)
(12, 121, 3, 5, 5, 1, 1/2)
(12, 122, 3, 5, 6, 1, 1/2)
(12, 123, 3, 5, 5, 2, 1)
(12, 124, 3, 5, 6, 0, 0)
(12, 125, 3, 5, 7, 2, 1)
(12, 126, 3, 5, 4, 14, 7)
(12, 127, 3, 5, 5, 2, 1)
(12, 128, 3, 5, 6, 1, 1/2)
(12, 129, 3, 5, 5, 2, 1)
(12, 130, 3, 5, 6, 0, 0)
(12, 131, 3, 5, 7, 0, 0)
(12, 132, 3, 5, 6, 2, 1)
(12, 1

...

(93, 145, 5, 5, 5, 3, 1)
(93, 146, 5, 5, 6, 1, 1/3)
(93, 147, 5, 5, 5, 3, 1)
(93, 148, 5, 5, 6, 1, 1/3)
(93, 149, 5, 5, 7, 1, 1/3)
(93, 150, 5, 5, 6, 3, 1)
(93, 151, 5, 5, 7, 0, 0)
(93, 152, 5, 5, 8, 2, 2/3)
(93, 153, 5, 5, 5, 96, 32)
(93, 154, 5, 5, 6, 4, 4/3)
(93, 155, 5, 5, 7, 1, 1/3)
(93, 156, 5, 5, 6, 3, 1)
(93, 157, 5, 5, 7, 0, 0)
(93, 158, 5, 5, 8, 1, 1/3)
(93, 159, 5, 5, 7, 3, 1)
(93, 160, 5, 5, 8, 0, 0)
(93, 161, 5, 5, 9, 4, 4/3)
(93, 162, 5, 5, 2, 282, 94)
(93, 163, 5, 5, 3, 4, 4/3)
(93, 164, 5, 5, 4, 1, 1/3)
(93, 165, 5, 5, 3, 3, 1)
(93, 166, 5, 5, 4, 0, 0)
(93, 167, 5, 5, 5, 1, 1/3)
(93, 168, 5, 5, 4, 3, 1)
(93, 169, 5, 5, 5, 3, 1)
(93, 170, 5, 5, 6, 2, 2/3)
(93, 171, 5, 5, 3, 96, 32)
(93, 172, 5, 5, 4, 3, 1)
(93, 173, 5, 5, 5, 0, 0)
(93, 174, 5, 5, 4, 3, 1)
(93, 175, 5, 5, 5, 3, 1)
(93, 176, 5, 5, 6, 3, 1)
(93, 177, 5, 5, 5, 3, 1)
(93, 178, 5, 5, 6, 0, 0)
(93, 179, 5, 5, 7, 3, 1)
(93, 180, 5, 5, 4, 96, 32)
(93, 181, 5, 5, 5, 3, 1)
(93, 182, 5, 5, 6, 1, 1/3)
(93, 183, 5, 5,

(95, 67, 5, 4, 5, 4, 4/5)
(95, 68, 5, 4, 6, 4, 4/5)
(95, 69, 5, 4, 5, 5, 1)
(95, 70, 5, 4, 6, 2, 2/5)
(95, 71, 5, 4, 7, 5, 1)
(95, 72, 5, 4, 4, 100, 20)
(95, 73, 5, 4, 5, 3, 3/5)
(95, 74, 5, 4, 6, 5, 1)
(95, 75, 5, 4, 5, 5, 1)
(95, 76, 5, 4, 6, 2, 2/5)
(95, 77, 5, 4, 7, 1, 1/5)
(95, 78, 5, 4, 6, 5, 1)
(95, 79, 5, 4, 7, 4, 4/5)
(95, 80, 5, 4, 8, 3, 3/5)
(95, 81, 5, 5, 1, 290, 58)
(95, 82, 5, 5, 2, 3, 3/5)
(95, 83, 5, 5, 3, 3, 3/5)
(95, 84, 5, 5, 2, 5, 1)
(95, 85, 5, 5, 3, 2, 2/5)
(95, 86, 5, 5, 4, 1, 1/5)
(95, 87, 5, 5, 3, 5, 1)
(95, 88, 5, 5, 4, 3, 3/5)
(95, 89, 5, 5, 5, 4, 4/5)
(95, 90, 5, 5, 2, 100, 20)
(95, 91, 5, 5, 3, 3, 3/5)
(95, 92, 5, 5, 4, 2, 2/5)
(95, 93, 5, 5, 3, 5, 1)
(95, 94, 5, 5, 4, 2, 2/5)
(95, 95, 5, 5, 5, 2, 2/5)
(95, 96, 5, 5, 4, 5, 1)
(95, 97, 5, 5, 5, 3, 3/5)
(95, 98, 5, 5, 6, 4, 4/5)
(95, 99, 5, 5, 3, 100, 20)
(95, 100, 5, 5, 4, 4, 4/5)
(95, 101, 5, 5, 5, 5, 1)
(95, 102, 5, 5, 4, 5, 1)
(95, 103, 5, 5, 5, 1, 1/5)
(95, 104, 5, 5, 6, 2, 2/5)
(95, 105, 5, 5, 5, 5, 1)


(96, 182, 5, 5, 6, 2, 1/2)
(96, 183, 5, 5, 5, 4, 1)
(96, 184, 5, 5, 6, 3, 3/4)
(96, 185, 5, 5, 7, 1, 1/4)
(96, 186, 5, 5, 6, 4, 1)
(96, 187, 5, 5, 7, 4, 1)
(96, 188, 5, 5, 8, 1, 1/4)
(96, 189, 5, 5, 3, 196, 49)
(96, 190, 5, 5, 4, 1, 1/4)
(96, 191, 5, 5, 5, 3, 3/4)
(96, 192, 5, 5, 4, 4, 1)
(96, 193, 5, 5, 5, 2, 1/2)
(96, 194, 5, 5, 6, 2, 1/2)
(96, 195, 5, 5, 5, 4, 1)
(96, 196, 5, 5, 6, 4, 1)
(96, 197, 5, 5, 7, 3, 3/4)
(96, 198, 5, 5, 4, 100, 25)
(96, 199, 5, 5, 5, 2, 1/2)
(96, 200, 5, 5, 6, 1, 1/4)
(96, 201, 5, 5, 5, 4, 1)
(96, 202, 5, 5, 6, 3, 3/4)
(96, 203, 5, 5, 7, 3, 3/4)
(97, 3, 5, 2, 1, 5, 1)
(97, 4, 5, 2, 2, 1, 1/5)
(97, 5, 5, 2, 3, 2, 2/5)
(97, 6, 5, 2, 2, 5, 1)
(97, 7, 5, 2, 3, 2, 2/5)
(97, 8, 5, 2, 4, 3, 3/5)
(97, 9, 5, 3, 1, 102, 102/5)
(97, 10, 5, 3, 2, 5, 1)
(97, 11, 5, 3, 3, 2, 2/5)
(97, 12, 5, 3, 2, 5, 1)
(97, 13, 5, 3, 3, 2, 2/5)
(97, 14, 5, 3, 4, 2, 2/5)
(97, 15, 5, 3, 3, 5, 1)
(97, 16, 5, 3, 4, 1, 1/5)
(97, 17, 5, 3, 5, 3, 3/5)
(97, 18, 5, 3, 2, 102, 102/5)
(97, 19, 5,

(98, 94, 5, 5, 4, 4, 2/3)
(98, 95, 5, 5, 5, 4, 2/3)
(98, 96, 5, 5, 4, 6, 1)
(98, 97, 5, 5, 5, 3, 1/2)
(98, 98, 5, 5, 6, 5, 5/6)
(98, 99, 5, 5, 3, 104, 52/3)
(98, 100, 5, 5, 4, 4, 2/3)
(98, 101, 5, 5, 5, 5, 5/6)
(98, 102, 5, 5, 4, 6, 1)
(98, 103, 5, 5, 5, 4, 2/3)
(98, 104, 5, 5, 6, 4, 2/3)
(98, 105, 5, 5, 5, 6, 1)
(98, 106, 5, 5, 6, 5, 5/6)
(98, 107, 5, 5, 7, 2, 1/3)
(98, 108, 5, 5, 2, 202, 101/3)
(98, 109, 5, 5, 3, 2, 1/3)
(98, 110, 5, 5, 4, 3, 1/2)
(98, 111, 5, 5, 3, 6, 1)
(98, 112, 5, 5, 4, 3, 1/2)
(98, 113, 5, 5, 5, 4, 2/3)
(98, 114, 5, 5, 4, 6, 1)
(98, 115, 5, 5, 5, 3, 1/2)
(98, 116, 5, 5, 6, 3, 1/2)
(98, 117, 5, 5, 3, 104, 52/3)
(98, 118, 5, 5, 4, 4, 2/3)
(98, 119, 5, 5, 5, 2, 1/3)
(98, 120, 5, 5, 4, 6, 1)
(98, 121, 5, 5, 5, 4, 2/3)
(98, 122, 5, 5, 6, 4, 2/3)
(98, 123, 5, 5, 5, 6, 1)
(98, 124, 5, 5, 6, 2, 1/3)
(98, 125, 5, 5, 7, 4, 2/3)
(98, 126, 5, 5, 4, 104, 52/3)
(98, 127, 5, 5, 5, 3, 1/2)
(98, 128, 5, 5, 6, 3, 1/2)
(98, 129, 5, 5, 5, 6, 1)
(98, 130, 5, 5, 6, 4, 2/3)
(98, 131, 

(100, 12, 5, 3, 2, 4, 1)
(100, 13, 5, 3, 3, 1, 1/4)
(100, 14, 5, 3, 4, 1, 1/4)
(100, 15, 5, 3, 3, 4, 1)
(100, 16, 5, 3, 4, 2, 1/2)
(100, 17, 5, 3, 5, 1, 1/4)
(100, 18, 5, 3, 2, 104, 26)
(100, 19, 5, 3, 3, 1, 1/4)
(100, 20, 5, 3, 4, 2, 1/2)
(100, 21, 5, 3, 3, 4, 1)
(100, 22, 5, 3, 4, 3, 3/4)
(100, 23, 5, 3, 5, 2, 1/2)
(100, 24, 5, 3, 4, 4, 1)
(100, 25, 5, 3, 5, 2, 1/2)
(100, 26, 5, 3, 6, 3, 3/4)
(100, 27, 5, 4, 1, 204, 51)
(100, 28, 5, 4, 2, 3, 3/4)
(100, 29, 5, 4, 3, 1, 1/4)
(100, 30, 5, 4, 2, 4, 1)
(100, 31, 5, 4, 3, 1, 1/4)
(100, 32, 5, 4, 4, 2, 1/2)
(100, 33, 5, 4, 3, 4, 1)
(100, 34, 5, 4, 4, 1, 1/4)
(100, 35, 5, 4, 5, 1, 1/4)
(100, 36, 5, 4, 2, 104, 26)
(100, 37, 5, 4, 3, 1, 1/4)
(100, 38, 5, 4, 4, 2, 1/2)
(100, 39, 5, 4, 3, 4, 1)
(100, 40, 5, 4, 4, 4, 1)
(100, 41, 5, 4, 5, 4, 1)
(100, 42, 5, 4, 4, 4, 1)
(100, 43, 5, 4, 5, 1, 1/4)
(100, 44, 5, 4, 6, 1, 1/4)
(100, 45, 5, 4, 3, 104, 26)
(100, 46, 5, 4, 4, 2, 1/2)
(100, 47, 5, 4, 5, 3, 3/4)
(100, 48, 5, 4, 4, 4, 1)
(100, 49, 5, 4, 5, 

In [36]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
dspk=1
for h in [0..len(s)-1]:
        k=s[h][0]
        dspk=digitsum(p,k)
        lk=len(k.digits(p))
        poly=D(s[h][1])
        for m in [3..203]:
            lm=len(m.digits(p))
            dsm=digitsum(p,m)
            polym=evaluate(poly,m)
            o=order(p,polym)
            rk=o/dspk
            rm=o/dsm
            print((k,m,lk,lm,dsm,o,(rk-order(p,m))*dspk))
                

(1, 3, 1, 2, 1, 1, 0)
(1, 4, 1, 2, 2, 0, 0)
(1, 5, 1, 2, 3, 0, 0)
(1, 6, 1, 2, 2, 1, 0)
(1, 7, 1, 2, 3, 0, 0)
(1, 8, 1, 2, 4, 0, 0)
(1, 9, 1, 3, 1, 2, 0)
(1, 10, 1, 3, 2, 0, 0)
(1, 11, 1, 3, 3, 0, 0)
(1, 12, 1, 3, 2, 1, 0)
(1, 13, 1, 3, 3, 0, 0)
(1, 14, 1, 3, 4, 0, 0)
(1, 15, 1, 3, 3, 1, 0)
(1, 16, 1, 3, 4, 0, 0)
(1, 17, 1, 3, 5, 0, 0)
(1, 18, 1, 3, 2, 2, 0)
(1, 19, 1, 3, 3, 0, 0)
(1, 20, 1, 3, 4, 0, 0)
(1, 21, 1, 3, 3, 1, 0)
(1, 22, 1, 3, 4, 0, 0)
(1, 23, 1, 3, 5, 0, 0)
(1, 24, 1, 3, 4, 1, 0)
(1, 25, 1, 3, 5, 0, 0)
(1, 26, 1, 3, 6, 0, 0)
(1, 27, 1, 4, 1, 3, 0)
(1, 28, 1, 4, 2, 0, 0)
(1, 29, 1, 4, 3, 0, 0)
(1, 30, 1, 4, 2, 1, 0)
(1, 31, 1, 4, 3, 0, 0)
(1, 32, 1, 4, 4, 0, 0)
(1, 33, 1, 4, 3, 1, 0)
(1, 34, 1, 4, 4, 0, 0)
(1, 35, 1, 4, 5, 0, 0)
(1, 36, 1, 4, 2, 2, 0)
(1, 37, 1, 4, 3, 0, 0)
(1, 38, 1, 4, 4, 0, 0)
(1, 39, 1, 4, 3, 1, 0)
(1, 40, 1, 4, 4, 0, 0)
(1, 41, 1, 4, 5, 0, 0)
(1, 42, 1, 4, 4, 1, 0)
(1, 43, 1, 4, 5, 0, 0)
(1, 44, 1, 4, 6, 0, 0)
(1, 45, 1, 4, 3, 2, 0)
(1, 46, 1, 4, 4, 0

(4, 170, 2, 5, 6, 1, 1)
(4, 171, 2, 5, 3, 6, 2)
(4, 172, 2, 5, 4, 1, 1)
(4, 173, 2, 5, 5, 0, 0)
(4, 174, 2, 5, 4, 2, 0)
(4, 175, 2, 5, 5, 0, 0)
(4, 176, 2, 5, 6, 0, 0)
(4, 177, 2, 5, 5, 2, 0)
(4, 178, 2, 5, 6, 0, 0)
(4, 179, 2, 5, 7, 1, 1)
(4, 180, 2, 5, 4, 6, 2)
(4, 181, 2, 5, 5, 1, 1)
(4, 182, 2, 5, 6, 0, 0)
(4, 183, 2, 5, 5, 2, 0)
(4, 184, 2, 5, 6, 0, 0)
(4, 185, 2, 5, 7, 0, 0)
(4, 186, 2, 5, 6, 2, 0)
(4, 187, 2, 5, 7, 0, 0)
(4, 188, 2, 5, 8, 1, 1)
(4, 189, 2, 5, 3, 10, 4)
(4, 190, 2, 5, 4, 1, 1)
(4, 191, 2, 5, 5, 0, 0)
(4, 192, 2, 5, 4, 2, 0)
(4, 193, 2, 5, 5, 0, 0)
(4, 194, 2, 5, 6, 0, 0)
(4, 195, 2, 5, 5, 2, 0)
(4, 196, 2, 5, 6, 0, 0)
(4, 197, 2, 5, 7, 1, 1)
(4, 198, 2, 5, 4, 6, 2)
(4, 199, 2, 5, 5, 1, 1)
(4, 200, 2, 5, 6, 0, 0)
(4, 201, 2, 5, 5, 2, 0)
(4, 202, 2, 5, 6, 0, 0)
(4, 203, 2, 5, 7, 0, 0)
(5, 3, 2, 2, 1, 3, 0)
(5, 4, 2, 2, 2, 1, 1)
(5, 5, 2, 2, 3, 1, 1)
(5, 6, 2, 2, 2, 3, 0)
(5, 7, 2, 2, 3, 3, 3)
(5, 8, 2, 2, 4, 2, 2)
(5, 9, 2, 3, 1, 8, 2)
(5, 10, 2, 3, 2, 2, 2)
(5, 11

(6, 142, 2, 5, 6, 3, 3)
(6, 143, 2, 5, 7, 1, 1)
(6, 144, 2, 5, 4, 8, 4)
(6, 145, 2, 5, 5, 1, 1)
(6, 146, 2, 5, 6, 1, 1)
(6, 147, 2, 5, 5, 2, 0)
(6, 148, 2, 5, 6, 2, 2)
(6, 149, 2, 5, 7, 2, 2)
(6, 150, 2, 5, 6, 2, 0)
(6, 151, 2, 5, 7, 1, 1)
(6, 152, 2, 5, 8, 1, 1)
(6, 153, 2, 5, 5, 8, 4)
(6, 154, 2, 5, 6, 1, 1)
(6, 155, 2, 5, 7, 2, 2)
(6, 156, 2, 5, 6, 2, 0)
(6, 157, 2, 5, 7, 1, 1)
(6, 158, 2, 5, 8, 1, 1)
(6, 159, 2, 5, 7, 2, 0)
(6, 160, 2, 5, 8, 1, 1)
(6, 161, 2, 5, 9, 1, 1)
(6, 162, 2, 5, 2, 20, 12)
(6, 163, 2, 5, 3, 1, 1)
(6, 164, 2, 5, 4, 1, 1)
(6, 165, 2, 5, 3, 2, 0)
(6, 166, 2, 5, 4, 1, 1)
(6, 167, 2, 5, 5, 1, 1)
(6, 168, 2, 5, 4, 2, 0)
(6, 169, 2, 5, 5, 2, 2)
(6, 170, 2, 5, 6, 1, 1)
(6, 171, 2, 5, 3, 8, 4)
(6, 172, 2, 5, 4, 1, 1)
(6, 173, 2, 5, 5, 1, 1)
(6, 174, 2, 5, 4, 2, 0)
(6, 175, 2, 5, 5, 2, 2)
(6, 176, 2, 5, 6, 2, 2)
(6, 177, 2, 5, 5, 2, 0)
(6, 178, 2, 5, 6, 1, 1)
(6, 179, 2, 5, 7, 1, 1)
(6, 180, 2, 5, 4, 8, 4)
(6, 181, 2, 5, 5, 1, 1)
(6, 182, 2, 5, 6, 4, 4)
(6, 183, 2, 5,

(8, 115, 2, 5, 5, 3, 3)
(8, 116, 2, 5, 6, 2, 2)
(8, 117, 2, 5, 3, 12, 4)
(8, 118, 2, 5, 4, 2, 2)
(8, 119, 2, 5, 5, 2, 2)
(8, 120, 2, 5, 4, 4, 0)
(8, 121, 2, 5, 5, 4, 4)
(8, 122, 2, 5, 6, 4, 4)
(8, 123, 2, 5, 5, 4, 0)
(8, 124, 2, 5, 6, 2, 2)
(8, 125, 2, 5, 7, 2, 2)
(8, 126, 2, 5, 4, 12, 4)
(8, 127, 2, 5, 5, 2, 2)
(8, 128, 2, 5, 6, 3, 3)
(8, 129, 2, 5, 5, 4, 0)
(8, 130, 2, 5, 6, 3, 3)
(8, 131, 2, 5, 7, 4, 4)
(8, 132, 2, 5, 6, 4, 0)
(8, 133, 2, 5, 7, 2, 2)
(8, 134, 2, 5, 8, 2, 2)
(8, 135, 2, 5, 3, 20, 8)
(8, 136, 2, 5, 4, 2, 2)
(8, 137, 2, 5, 5, 2, 2)
(8, 138, 2, 5, 4, 4, 0)
(8, 139, 2, 5, 5, 4, 4)
(8, 140, 2, 5, 6, 3, 3)
(8, 141, 2, 5, 5, 4, 0)
(8, 142, 2, 5, 6, 4, 4)
(8, 143, 2, 5, 7, 2, 2)
(8, 144, 2, 5, 4, 12, 4)
(8, 145, 2, 5, 5, 2, 2)
(8, 146, 2, 5, 6, 2, 2)
(8, 147, 2, 5, 5, 4, 0)
(8, 148, 2, 5, 6, 4, 4)
(8, 149, 2, 5, 7, 5, 5)
(8, 150, 2, 5, 6, 4, 0)
(8, 151, 2, 5, 7, 2, 2)
(8, 152, 2, 5, 8, 2, 2)
(8, 153, 2, 5, 5, 12, 4)
(8, 154, 2, 5, 6, 2, 2)
(8, 155, 2, 5, 7, 3, 3)
(8, 156, 2,

(11, 36, 3, 4, 2, 14, 8)
(11, 37, 3, 4, 3, 2, 2)
(11, 38, 3, 4, 4, 1, 1)
(11, 39, 3, 4, 3, 3, 0)
(11, 40, 3, 4, 4, 2, 2)
(11, 41, 3, 4, 5, 2, 2)
(11, 42, 3, 4, 4, 3, 0)
(11, 43, 3, 4, 5, 1, 1)
(11, 44, 3, 4, 6, 2, 2)
(11, 45, 3, 4, 3, 14, 8)
(11, 46, 3, 4, 4, 3, 3)
(11, 47, 3, 4, 5, 2, 2)
(11, 48, 3, 4, 4, 3, 0)
(11, 49, 3, 4, 5, 1, 1)
(11, 50, 3, 4, 6, 1, 1)
(11, 51, 3, 4, 5, 3, 0)
(11, 52, 3, 4, 6, 1, 1)
(11, 53, 3, 4, 7, 3, 3)
(11, 54, 3, 4, 2, 25, 16)
(11, 55, 3, 4, 3, 3, 3)
(11, 56, 3, 4, 4, 1, 1)
(11, 57, 3, 4, 3, 3, 0)
(11, 58, 3, 4, 4, 1, 1)
(11, 59, 3, 4, 5, 1, 1)
(11, 60, 3, 4, 4, 3, 0)
(11, 61, 3, 4, 5, 2, 2)
(11, 62, 3, 4, 6, 3, 3)
(11, 63, 3, 4, 3, 14, 8)
(11, 64, 3, 4, 4, 2, 2)
(11, 65, 3, 4, 5, 1, 1)
(11, 66, 3, 4, 4, 3, 0)
(11, 67, 3, 4, 5, 2, 2)
(11, 68, 3, 4, 6, 2, 2)
(11, 69, 3, 4, 5, 3, 0)
(11, 70, 3, 4, 6, 1, 1)
(11, 71, 3, 4, 7, 2, 2)
(11, 72, 3, 4, 4, 14, 8)
(11, 73, 3, 4, 5, 5, 5)
(11, 74, 3, 4, 6, 2, 2)
(11, 75, 3, 4, 5, 3, 0)
(11, 76, 3, 4, 6, 1, 1)
(11, 77, 3

...

(94, 39, 5, 4, 3, 4, 0)
(94, 40, 5, 4, 4, 4, 4)
(94, 41, 5, 4, 5, 4, 4)
(94, 42, 5, 4, 4, 4, 0)
(94, 43, 5, 4, 5, 0, 0)
(94, 44, 5, 4, 6, 2, 2)
(94, 45, 5, 4, 3, 98, 90)
(94, 46, 5, 4, 4, 3, 3)
(94, 47, 5, 4, 5, 3, 3)
(94, 48, 5, 4, 4, 4, 0)
(94, 49, 5, 4, 5, 0, 0)
(94, 50, 5, 4, 6, 1, 1)
(94, 51, 5, 4, 5, 4, 0)
(94, 52, 5, 4, 6, 1, 1)
(94, 53, 5, 4, 7, 1, 1)
(94, 54, 5, 4, 2, 192, 180)
(94, 55, 5, 4, 3, 1, 1)
(94, 56, 5, 4, 4, 1, 1)
(94, 57, 5, 4, 3, 4, 0)
(94, 58, 5, 4, 4, 0, 0)
(94, 59, 5, 4, 5, 0, 0)
(94, 60, 5, 4, 4, 4, 0)
(94, 61, 5, 4, 5, 1, 1)
(94, 62, 5, 4, 6, 3, 3)
(94, 63, 5, 4, 3, 98, 90)
(94, 64, 5, 4, 4, 3, 3)
(94, 65, 5, 4, 5, 0, 0)
(94, 66, 5, 4, 4, 4, 0)
(94, 67, 5, 4, 5, 3, 3)
(94, 68, 5, 4, 6, 3, 3)
(94, 69, 5, 4, 5, 4, 0)
(94, 70, 5, 4, 6, 0, 0)
(94, 71, 5, 4, 7, 4, 4)
(94, 72, 5, 4, 4, 98, 90)
(94, 73, 5, 4, 5, 2, 2)
(94, 74, 5, 4, 6, 3, 3)
(94, 75, 5, 4, 5, 4, 0)
(94, 76, 5, 4, 6, 1, 1)
(94, 77, 5, 4, 7, 0, 0)
(94, 78, 5, 4, 6, 4, 0)
(94, 79, 5, 4, 7, 1, 1)
(94, 8

(95, 174, 5, 5, 4, 5, 0)
(95, 175, 5, 5, 5, 4, 4)
(95, 176, 5, 5, 6, 4, 4)
(95, 177, 5, 5, 5, 5, 0)
(95, 178, 5, 5, 6, 2, 2)
(95, 179, 5, 5, 7, 4, 4)
(95, 180, 5, 5, 4, 100, 90)
(95, 181, 5, 5, 5, 4, 4)
(95, 182, 5, 5, 6, 3, 3)
(95, 183, 5, 5, 5, 5, 0)
(95, 184, 5, 5, 6, 1, 1)
(95, 185, 5, 5, 7, 1, 1)
(95, 186, 5, 5, 6, 5, 0)
(95, 187, 5, 5, 7, 5, 5)
(95, 188, 5, 5, 8, 2, 2)
(95, 189, 5, 5, 3, 195, 180)
(95, 190, 5, 5, 4, 2, 2)
(95, 191, 5, 5, 5, 4, 4)
(95, 192, 5, 5, 4, 5, 0)
(95, 193, 5, 5, 5, 2, 2)
(95, 194, 5, 5, 6, 1, 1)
(95, 195, 5, 5, 5, 5, 0)
(95, 196, 5, 5, 6, 5, 5)
(95, 197, 5, 5, 7, 4, 4)
(95, 198, 5, 5, 4, 100, 90)
(95, 199, 5, 5, 5, 3, 3)
(95, 200, 5, 5, 6, 2, 2)
(95, 201, 5, 5, 5, 5, 0)
(95, 202, 5, 5, 6, 3, 3)
(95, 203, 5, 5, 7, 3, 3)
(96, 3, 5, 2, 1, 4, 0)
(96, 4, 5, 2, 2, 1, 1)
(96, 5, 5, 2, 3, 2, 2)
(96, 6, 5, 2, 2, 4, 0)
(96, 7, 5, 2, 3, 2, 2)
(96, 8, 5, 2, 4, 3, 3)
(96, 9, 5, 3, 1, 100, 92)
(96, 10, 5, 3, 2, 5, 5)
(96, 11, 5, 3, 3, 2, 2)
(96, 12, 5, 3, 2, 4, 0)
(96,

(97, 105, 5, 5, 5, 5, 0)
(97, 106, 5, 5, 6, 4, 4)
(97, 107, 5, 5, 7, 1, 1)
(97, 108, 5, 5, 2, 199, 184)
(97, 109, 5, 5, 3, 1, 1)
(97, 110, 5, 5, 4, 2, 2)
(97, 111, 5, 5, 3, 5, 0)
(97, 112, 5, 5, 4, 1, 1)
(97, 113, 5, 5, 5, 2, 2)
(97, 114, 5, 5, 4, 5, 0)
(97, 115, 5, 5, 5, 2, 2)
(97, 116, 5, 5, 6, 2, 2)
(97, 117, 5, 5, 3, 102, 92)
(97, 118, 5, 5, 4, 3, 3)
(97, 119, 5, 5, 5, 1, 1)
(97, 120, 5, 5, 4, 5, 0)
(97, 121, 5, 5, 5, 2, 2)
(97, 122, 5, 5, 6, 2, 2)
(97, 123, 5, 5, 5, 5, 0)
(97, 124, 5, 5, 6, 1, 1)
(97, 125, 5, 5, 7, 3, 3)
(97, 126, 5, 5, 4, 102, 92)
(97, 127, 5, 5, 5, 2, 2)
(97, 128, 5, 5, 6, 2, 2)
(97, 129, 5, 5, 5, 5, 0)
(97, 130, 5, 5, 6, 2, 2)
(97, 131, 5, 5, 7, 1, 1)
(97, 132, 5, 5, 6, 5, 0)
(97, 133, 5, 5, 7, 2, 2)
(97, 134, 5, 5, 8, 1, 1)
(97, 135, 5, 5, 3, 199, 184)
(97, 136, 5, 5, 4, 1, 1)
(97, 137, 5, 5, 5, 4, 4)
(97, 138, 5, 5, 4, 5, 0)
(97, 139, 5, 5, 5, 2, 2)
(97, 140, 5, 5, 6, 3, 3)
(97, 141, 5, 5, 5, 5, 0)
(97, 142, 5, 5, 6, 3, 3)
(97, 143, 5, 5, 7, 3, 3)
(97, 144, 5

(99, 32, 5, 4, 4, 2, 2)
(99, 33, 5, 4, 3, 3, 0)
(99, 34, 5, 4, 4, 1, 1)
(99, 35, 5, 4, 5, 1, 1)
(99, 36, 5, 4, 2, 102, 96)
(99, 37, 5, 4, 3, 1, 1)
(99, 38, 5, 4, 4, 2, 2)
(99, 39, 5, 4, 3, 3, 0)
(99, 40, 5, 4, 4, 4, 4)
(99, 41, 5, 4, 5, 4, 4)
(99, 42, 5, 4, 4, 3, 0)
(99, 43, 5, 4, 5, 1, 1)
(99, 44, 5, 4, 6, 1, 1)
(99, 45, 5, 4, 3, 102, 96)
(99, 46, 5, 4, 4, 2, 2)
(99, 47, 5, 4, 5, 3, 3)
(99, 48, 5, 4, 4, 3, 0)
(99, 49, 5, 4, 5, 1, 1)
(99, 50, 5, 4, 6, 2, 2)
(99, 51, 5, 4, 5, 3, 0)
(99, 52, 5, 4, 6, 2, 2)
(99, 53, 5, 4, 7, 1, 1)
(99, 54, 5, 4, 2, 201, 192)
(99, 55, 5, 4, 3, 1, 1)
(99, 56, 5, 4, 4, 3, 3)
(99, 57, 5, 4, 3, 3, 0)
(99, 58, 5, 4, 4, 3, 3)
(99, 59, 5, 4, 5, 2, 2)
(99, 60, 5, 4, 4, 3, 0)
(99, 61, 5, 4, 5, 1, 1)
(99, 62, 5, 4, 6, 1, 1)
(99, 63, 5, 4, 3, 102, 96)
(99, 64, 5, 4, 4, 2, 2)
(99, 65, 5, 4, 5, 5, 5)
(99, 66, 5, 4, 4, 3, 0)
(99, 67, 5, 4, 5, 4, 4)
(99, 68, 5, 4, 6, 6, 6)
(99, 69, 5, 4, 5, 3, 0)
(99, 70, 5, 4, 6, 1, 1)
(99, 71, 5, 4, 7, 3, 3)
(99, 72, 5, 4, 4, 102, 96)


(100, 156, 5, 5, 6, 4, 0)
(100, 157, 5, 5, 7, 1, 1)
(100, 158, 5, 5, 8, 2, 2)
(100, 159, 5, 5, 7, 4, 0)
(100, 160, 5, 5, 8, 1, 1)
(100, 161, 5, 5, 9, 4, 4)
(100, 162, 5, 5, 2, 304, 288)
(100, 163, 5, 5, 3, 4, 4)
(100, 164, 5, 5, 4, 2, 2)
(100, 165, 5, 5, 3, 4, 0)
(100, 166, 5, 5, 4, 1, 1)
(100, 167, 5, 5, 5, 2, 2)
(100, 168, 5, 5, 4, 4, 0)
(100, 169, 5, 5, 5, 4, 4)
(100, 170, 5, 5, 6, 5, 5)
(100, 171, 5, 5, 3, 104, 96)
(100, 172, 5, 5, 4, 4, 4)
(100, 173, 5, 5, 5, 1, 1)
(100, 174, 5, 5, 4, 4, 0)
(100, 175, 5, 5, 5, 3, 3)
(100, 176, 5, 5, 6, 3, 3)
(100, 177, 5, 5, 5, 4, 0)
(100, 178, 5, 5, 6, 3, 3)
(100, 179, 5, 5, 7, 2, 2)
(100, 180, 5, 5, 4, 104, 96)
(100, 181, 5, 5, 5, 1, 1)
(100, 182, 5, 5, 6, 1, 1)
(100, 183, 5, 5, 5, 4, 0)
(100, 184, 5, 5, 6, 2, 2)
(100, 185, 5, 5, 7, 3, 3)
(100, 186, 5, 5, 6, 4, 0)
(100, 187, 5, 5, 7, 3, 3)
(100, 188, 5, 5, 8, 1, 1)
(100, 189, 5, 5, 3, 204, 192)
(100, 190, 5, 5, 4, 1, 1)
(100, 191, 5, 5, 5, 2, 2)
(100, 192, 5, 5, 4, 4, 0)
(100, 193, 5, 5, 5, 2, 2

In [62]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        if k==1:
            #dspk=digitsum(p,k)
            #lk=len(k.digits(p))
            poly=D(s[h][1])
            for m in [3..203]:
                opm=order(p,m)
                lm=len(m.digits(p))
                dsm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rk=o
                rm=o/dsm
                print((k,m,lk,lm,dsm,opm-rk))
                data=data+[opm-rk]
print(data)               

(1, 3, 5, 2, 1, 0)
(1, 4, 5, 2, 2, 0)
(1, 5, 5, 2, 3, 0)
(1, 6, 5, 2, 2, 0)
(1, 7, 5, 2, 3, 0)
(1, 8, 5, 2, 4, 0)
(1, 9, 5, 3, 1, 0)
(1, 10, 5, 3, 2, 0)
(1, 11, 5, 3, 3, 0)
(1, 12, 5, 3, 2, 0)
(1, 13, 5, 3, 3, 0)
(1, 14, 5, 3, 4, 0)
(1, 15, 5, 3, 3, 0)
(1, 16, 5, 3, 4, 0)
(1, 17, 5, 3, 5, 0)
(1, 18, 5, 3, 2, 0)
(1, 19, 5, 3, 3, 0)
(1, 20, 5, 3, 4, 0)
(1, 21, 5, 3, 3, 0)
(1, 22, 5, 3, 4, 0)
(1, 23, 5, 3, 5, 0)
(1, 24, 5, 3, 4, 0)
(1, 25, 5, 3, 5, 0)
(1, 26, 5, 3, 6, 0)
(1, 27, 5, 4, 1, 0)
(1, 28, 5, 4, 2, 0)
(1, 29, 5, 4, 3, 0)
(1, 30, 5, 4, 2, 0)
(1, 31, 5, 4, 3, 0)
(1, 32, 5, 4, 4, 0)
(1, 33, 5, 4, 3, 0)
(1, 34, 5, 4, 4, 0)
(1, 35, 5, 4, 5, 0)
(1, 36, 5, 4, 2, 0)
(1, 37, 5, 4, 3, 0)
(1, 38, 5, 4, 4, 0)
(1, 39, 5, 4, 3, 0)
(1, 40, 5, 4, 4, 0)
(1, 41, 5, 4, 5, 0)
(1, 42, 5, 4, 4, 0)
(1, 43, 5, 4, 5, 0)
(1, 44, 5, 4, 6, 0)
(1, 45, 5, 4, 3, 0)
(1, 46, 5, 4, 4, 0)
(1, 47, 5, 4, 5, 0)
(1, 48, 5, 4, 4, 0)
(1, 49, 5, 4, 5, 0)
(1, 50, 5, 4, 6, 0)
(1, 51, 5, 4, 5, 0)
(1, 52, 5, 4, 6, 0)
(1, 53,

In [139]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        if k==1:
            dspk=digitsum(p,k)
            lk=len(k.digits(p))
            poly=D(s[h][1])
            for m in [3..1003]:
                opm=order(p,m)
                lm=len(m.digits(p))
                dsm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rm=o/dsm
                data=data+[k*opm-o]
print(uniteList(data))           

[0]


In [140]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        if k==p:
            dk=digitsum(p,k)
            lk=len(k.digits(p))
            poly=D(s[h][1])
            for m in [3..1003]:
                mp=ZZ(mod(m,p))
                opm=order(p,m)
                lm=len(m.digits(p))
                dm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rm=o/dm
                rk=o/dk
                if mp==0:
                    data=data+[k*opm-o]
uld=uniteList(data);print(uld)

[2]


In [141]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        if k==p^2:
            dk=digitsum(p,k)
            lk=len(k.digits(p))
            poly=D(s[h][1])
            for m in [3..203]:
                mp=ZZ(mod(m,p))
                opm=order(p,m)
                lm=len(m.digits(p))
                dm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rm=o/dm
                rk=o/dk
                if mp==0:
                    data=data+[k*opm-o]
print(uniteList(data))

[8]


In [142]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        if k==p^3:
            dk=digitsum(p,k)
            lk=len(k.digits(p))
            poly=D(s[h][1])
            for m in [3..203]:
                mp=ZZ(mod(m,p))
                opm=order(p,m)
                lm=len(m.digits(p))
                dm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rm=o/dm
                rk=o/dk
                if mp==0:
                    data=data+[k*opm-o]
print(uniteList(data))

[26]


In [143]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        if k==p^4:
            dk=digitsum(p,k)
            lk=len(k.digits(p))
            poly=D(s[h][1])
            for m in [3..203]:
                mp=ZZ(mod(m,p))
                opm=order(p,m)
                lm=len(m.digits(p))
                dm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rm=o/dm
                rk=o/dk
                if mp==0:
                    data=data+[k*opm-o]
print(uniteList(data))

[80]


In [ ]:
# I only have data for k <=99 or 100, so I can't go beyond p^4 for p = 3.
# Several OEIS sequences begin with 0,2,8,26,80.

In [148]:
import pickle
rfile = open('/Users/barrybrent/5dec22no1.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
p=3
data=[]
for h in [0..len(s)-1]:
        k=s[h][0]
        dk=digitsum(p,k)
        if dk==2:
            poly=D(s[h][1])
            for m in [3..1003]:
                mp=ZZ(mod(m,p))
                opm=order(p,m)
                lm=len(m.digits(p))
                dm=digitsum(p,m)
                polym=evaluate(poly,m)
                o=order(p,polym)
                rm=o/dm
                rk=o/dk
                if mp==0:
                    data=data+[k*opm+dk-o]
print(uniteList(data)) # OEIS A055 235: Sums of two powers of 3
# but which 'two powers'?

[2, 4, 6, 10, 12, 18, 28, 30, 36, 54, 82, 84, 90]


In [152]:
lst=[2, 4, 6, 10, 12, 18, 28, 30, 36, 54, 82, 84, 90]
for j in lst:
    print(lst[j].digits(3))

[0, 2]
[0, 1, 1]
[1, 0, 0, 1]
[1, 0, 0, 0, 1]
[0, 0, 1, 0, 1]


IndexError: list index out of range